In [1]:
import os
os.chdir('/home')

from lib import labels

FINDING: 0.4% of labels have no associated text

maybe we can infer these later by using the graph and the vector space?

In [2]:
word_to_glove = labels.load_embedding()
X = labels.embedding_matrix('train', -1, word_to_glove)
A = labels.adjacency_matrix('train', -1, identity=True)
print(X.shape, A.shape)

Starting load_embedding...
load_embedding took 48.32120418548584 seconds
Starting embedding_matrix...
embedding_matrix took 8.695425987243652 seconds
Starting adjacency_matrix...
adjacency_matrix took 28.31315565109253 seconds
(3862, 300) (3862, 3862)


In [3]:
# classify some labels by clustering the glove space
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline


'''To find the best K
data = []
for n_clusters in [5, 10, 20, 40, 60, 80, 100]:
    clusterer = KMeans(n_clusters)
    clusterer.fit(X)
    clusters = clusterer.predict(X)
    data.append([n_clusters, silhouette_score(X, clusters)])

data = np.array(data)
plt.plot(data[:,0], data[:,1], 'o-')
'''

n_clusters = 20 # the elbow point of the cluster learning curve
clusterer = KMeans(n_clusters)
clusterer.fit(X)
clusters = clusterer.predict(X)
sample_indices = np.array([np.random.choice(np.where(clusters==i)[0])
                            for i in range(n_clusters)])
onehot_clusters = np.zeros((X.shape[0], n_clusters))
onehot_clusters[sample_indices, np.arange(n_clusters)] = 1

This clustering is bad. It clusters according to the features; should use clustering according to the graph architecture, such as modularity-based clustering.

The vanilla GCN won't work because it only operates with full gradient descent, not batches. Keras can't really do that.

In [4]:
def normalize_adjacency(A):
    D = A.sum(axis=1)
    D_pow = np.power(D, -1/2) * np.eye(A.shape[0])
    return np.linalg.multi_dot([D_pow, A, D_pow])
A_norm = normalize_adjacency(A)

In [1]:
from tensorflow.keras.layers import Input, Dense, Activation, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

def GraphConvLayer(A_norm, size):
    def graphconv(in_layer, activation):
        out = Dense(size, batch_size=A_norm.shape[0])(in_layer)
        out = Lambda(lambda X: K.dot(A_norm, X))(out)
        out = Activation(activation)(out)
        return out
    return graphconv

def GCN(n_nodes, layer_sizes, n_classes):
    X_input = Input((300,))
    Anorm_input = Input((n_nodes,))
    latent = GraphConvLayer(Anorm_input, layer_sizes[0])(X_input, activation='relu')
    for size in layer_sizes[1:]:
        latent = GraphConvLayer(Anorm_input, size)(latent, activation='relu')
    out = GraphConvLayer(Anorm_input, n_classes)(latent, activation='softmax')
    
    model = Model([X_input, Anorm_input], out)
    latent_model = Model([X_input, Anorm_input], latent)
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    latent_model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model, latent_model

In [26]:
n_nodes = A.shape[0]
model, latent_model = GCN(n_nodes, [128, 64, 32], n_clusters)

In [27]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 300)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 128)               38528     
_________________________________________________________________
lambda_16 (Lambda)           (None, 128)               0         
_________________________________________________________________
activation_16 (Activation)   (None, 128)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 64)                8256      
_________________________________________________________________
lambda_17 (Lambda)           (None, 64)                0         
_________________________________________________________________
activation_17 (Activation)   (None, 64)                0         
__________

In [24]:
model.fit([X, A_norm], onehot_clusters, batch_size=A.shape[0], epochs=1)

Epoch 1/1
3862/3862 [==============================] - 0s 30us/step - loss: 0.0155


In [23]:
X2 = np.random.random((3862, 300))
A_norm2 = np.random.random((3862, 3862))
model.predict([X2, A_norm2], batch_size=3862).shape

InvalidArgumentError: Matrix size-incompatible: In[0]: [32,3862], In[1]: [32,128]
	 [[Node: lambda_12/MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](_arg_input_8_0_1/_275, dense_12/BiasAdd)]]
	 [[Node: activation_15/Softmax/_277 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_50_activation_15/Softmax", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

In [22]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 300)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 128)               38528     
_________________________________________________________________
lambda_12 (Lambda)           (None, 128)               0         
_________________________________________________________________
activation_12 (Activation)   (None, 128)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 64)                8256      
_________________________________________________________________
lambda_13 (Lambda)           (None, 64)                0         
_________________________________________________________________
activation_13 (Activation)   (None, 64)                0         
__________

In [13]:
from tensorflow.keras.layers import Input, Lambda
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model

X2_ = np.random.random((3862, 300))
A2_ = np.random.random((3862, 3862))
X2 = Input((300,))
A2 = Input((3862,))
out = Lambda(lambda X: K.dot(A2, X))(X2)
model = Model([X2, A2], out)
model.compile(loss='categorical_crossentropy', optimizer='sgd')

In [15]:
model.predict([X2_, A2_], batch_size=3862).shape

(3862, 300)